<p style="font-weight:bold;"> <span style="font-size: 36px"> IFRS 17 Calculation Engine - an Overview</span> </p>

# Introduction

Systemorph's IFRS 17 Calculation Engine is a tool and a solution to conduct all the essential IFRS 17 calculations based on some key input: modeled future **cashflows** and actual amounts for **groups of insurance contracts** and other relevant parameters (Yield Curve, FX rates, etc...). The output is a set of calculated results (Contractual Service Margin, Loss Component, Financial Performance, etc...) that are needed for IFRS 17 reporting, analysis, accounting, steering and other management information.

This notebook provides an **overview** of Systemorph's IFRS 17 Calculation Engine with brief introductions to all the main steps.

There are numerous **links** to several notebooks of the Calculation Engine. These notebooks contain specifications, code and also some detailed explanations with formulas in markdown blocks. The most important ones in this respect are [DataStructure](./DataModel/DataStructure#data-structure), [ImportScopes](./Import/ImportScopeCalculation#import-scope-calculation) and [ReportScopes](./Report/ReportScopes#report-scopes).

<a id='ifrs17-intro'></a>
# IFRS 17

IFRS 17 is a new accounting standard for insurance contracts. The previous standard IFRS 4, which is based on nominal accounting, is replaced by the economic accounting standard IFRS 17.

Some key points are repeated here briefly.

IFRS 17 deals with insurance contracts which are on the liability side of the balance sheet.  Other accounting standards deal with other parts of the business, primarily IFRS 9 for invested assets.

Insurance policies need to be allocated to homogeneous groups of insurance contracts (GICs) according to high-level principles established by the IFRS 17 standard, such as contract type, line of business, annual cohort etc. Insurers have some degree of freedom to optmize those GICs, as long as they respect the high-level principles.

Insurance contracts generate different types of cashflows. For example, **Amount Type** is a dimension refering to premiums, claims, expenses of different kinds, etc. for these cashflows. The Calculation Engine presented here expects input files of all the projected cashflows for each GIC, for different regular time points (quarterly or monthly, ...) and corresponding actual amounts observed for a given period. 

Each cashflow is characterized by all the necessary attributes: business unit, start time, amount type, novelty of business etc. and projected cashflow amounts in a regular time grid until the final run-off of all contracts.

If the entered cashflows are nominal (the usual case), the the present value (PV) of each cashflow is computed using proper yield curves for discounting.

A central element of IFRS 17 is the Analysis of Change (AoC).  The present value of the business (future cashflows) changes from the beginning to the end of the accounting period due to different effects, each of them constituting a modeled AoC step.

In IFRS 17, the value of future profitable business has a component called the Contractual Service Margin (CSM), defined at GIC level. The profit measured as CSM is recognized slowly over time rather than immediately in financial performance statements. On the other hand, a GIC can be onerous and produce a Loss Component (LC) instead, which needs to be recognized immediately.

Several reports on the valuation of cashflows, balance sheet and financial performance can be produced, each dealing with different aspects, such as
 - Present Value of Future Cash Flows (PVFCF)
 - Contractual Service Margin (CSM) and Loss Component (LC)
 - Liability for Remaining Coverage (LRC)
 - Liability for Incurred Claims (LIC)
 - Financial Performance
 - Accounting Events

<a id='data-input'></a>
# Data Model, Data Import

The IFRS 17 calculations are based on a well-defined [DataModel](./DataModel/DataStructure#data-structure).  The data input files are set up according to the data model, and the data import and all the ensuing calculations up to the final report generation respect the data model.

The IFRS 17 calculations start by importing data input files, and the more complex IFRS 17 Business Logic is defined in the [ImportScopes](./Import/ImportScopeCalculation#import-scope-calculation) notebook. This basis will later be used to calculate and generate different reports.

<a id='data-model'></a>
## Data Model

The data model includes the definition of reporting nodes and dimensions to be used, including identities, amount types, group of insurance contracts and many more.

The notebook [DataStructure](./DataModel/DataStructure#data-structure) defines the data records, dimensions and interfaces of the data model. There is a long section on the many [dimensions](./DataModel/DataStructure#dimensions) used to characterize the business and its IFRS 17 model.

The [input files](#data-input-dim-nodes) for ReportingNodes and especially Dimensions provide an overview of data structures actually used. Some dimensions are hierarchical, so some entries have parent entries.

Many abbreviations and acronyms used later in the documentation and the code are the **system names**, where the corresponding display names can be found in the corresponding input files together with the attributes.

<a id='calc-import'></a>
## Data import and related calculations

Several notebooks organize the data import.  Methods for importing and accessing data are in the [Importers](./Import/Importers) notebook. The [ImportStorage](./Import/ImportStorage#import-storage) prepare the data to perfom the calculation required.  Finally, methods for the calculations are in [ImportScopeCalculation](./Import/ImportScopeCalculation#import-scope-calculation).

However, the import as such is conducted by other notebooks which use the methods of Importers, ImportStorage and ImportScopeCalculation in a chain of notebook imports:

ImportStorage --> ImportScopeCalculation --> Importers

The calculations of ImportScopeCalculation cover a large part of the IFRS 17 methodology and are explained in the section below.

<a id='model-calc'></a>
# IFRS 17 Model Calculations

The notebook **[ImportScopeCalculation](./Import/ImportScopeCalculation#import-scope-calculation)** has all the calculations to be executed upon data import.  The methods used are explained there with formulas.  The centerpiece is the Analysis of Change (AoC) of the insurance business over one period (here: quarter).  The formulas often use acronyms (system names) as listed in the input file *DimensionsAndPartitions*.

The main concepts and parts of ImportScopeCalculation are listed here, with links.
- Structure and sequence of the **[Analysis of Change (AoC)](./Import/ImportScopeCalculation#aoc-step-structure)** steps for the given period
$$
$$
- **[Discounting](./Import/ImportScopeCalculation#discounting-calculation)** projected cash flows using [yield curves](#data-input-parameters) (current or locked in)
$$
$$
- **[Nominal](./Import/ImportScopeCalculation#nominal-values)** cash flow amounts per AoC step
$$
$$
- **[Present Values (PV)](./Import/ImportScopeCalculation#present-values)** of cash flow amounts for all AoC steps:
    - [Cumulated discounted](./Import/ImportScopeCalculation#cumulated-discounted-cashflows) cash flow amounts for all AoC steps
    - [Telescoping difference](./Import/ImportScopeCalculation#telescopic-difference) calculation of cumulated discounted cashflows per AoC step, where the total PV change over the period is the telescoping sum of all these differences
    - [Present Value calculation](./Import/ImportScopeCalculation#present-value) at the beginning and end of the period, with its underlying [Present value base](./Import/ImportScopeCalculation#present-value-base) calculations for different AoC types and for [current and locked-in](./Import/ImportScopeCalculation#current-and-locked) yield curves
    - [Interest accretion](./Import/ImportScopeCalculation#interest-accretion) cash flow calculation (one of the AoC steps)
- **[Risk Adjustment (RA)](./Import/ImportScopeCalculation#risk-adjustment)** calculation, a requirement of the IFRS 17 standard
$$
$$
- **[Amortization](./Import/ImportScopeCalculation#amortization)**, the AoC step dealing with the reduction of the remaining exposure over the period:
    - [Coverage Unit (CU)](./Import/ImportScopeCalculation#coverage-units), the quantity of services provided by a given GIC, considering one period (reflecting the run-off pattern of the GIC)
    - [Amortization factor](./Import/ImportScopeCalculation#amortization-factor), the relative size of the amortization of a GIC, monthly and for a longer time interval
$$
$$
- **[Actuals](./Import/ImportScopeCalculation#actual-values)**, treatment of actual values per GIC or GRIC, distinguished according to by their Estimate Type: Ordinary Actuals, Advance Actuals, Overdue Actuals and Deferrable Actuals
    - Calculation of [Actuals Base](./Import/ImportScopeCalculation#actual-base) values for a given GIC and per Estimate Type and Amount Type
    - Different Estimate Types of Actuals and their calculations: Ordinary [Actuals](./Import/ImportScopeCalculation#actuals), [Advance Actuals](./Import/ImportScopeCalculation#advance-actuals) paid before the due date which is inside the AoC period, [Overdue Actuals](./Import/ImportScopeCalculation#overdue-actuals) paid after the AoC period, [Deferrable Actuals](./Import/ImportScopeCalculation#overdue-actuals) whose recognition is deferred (typically used for expenses of different kinds)

$$
$$
- **[Contractual Service Margin (CSM), Loss Component LC) and Loss Recovery Component (LRC)](./Import/ImportScopeCalculation#csm)**, modeling the treatment of unearned profits (CSM) and losses (LC, LRC) as required by the IFRS 17 standard, per GIC or GRIC:
    - [Fulfillment Cashflow (FCF)](./Import/ImportScopeCalculation#fulfillment-cashflow), same as the aforementioned cumulated discounted cashflow, now adding the risk adjustment component as a basis for CSM and LC calculations
    - [Technical Margin](./Import/ImportScopeCalculation#technical-margin), which is the FCF with special rules for different AoC steps including the Amortization step, also using and calculating the Interest Accretion Factor, Premiums, Deferrals and Investment Claims
    - [Switch Logic for CSM and LC](./Import/ImportScopeCalculation#technical-margin), switching between CSM and LC for a given AoC step in the AoC period; one of the two is always zero depending on the sign of the Technical Margin; distinguishing the [gross case](./Import/ImportScopeCalculation#gross-case) (no reinsurance) from the [reinsurance case](./Import/ImportScopeCalculation#reinsurance-case) which applies reinsurance coverage weights

<a id='reports'></a>
# Reports

Several types of reports can be generated from the same model basis.  The notebook **[ReportScopes](./Report/ReportScopes#report-scopes)** has many methods to produce different reports to be executed after the end of all data imports.

The reports are generated with the help of **DataCube**, a Systemorph tool that allows for the targeted retrieval of information from a large multi-dimensional (in-memory) database, selecting the desired categories, items and degrees of granularity and aggregation across different dimensions.

Some reports of ReportScopes just present existing data from the database, namely the imported data and the results already calculated using the methods of [ImportScopes](#model-calc).

Some other reports related to **[Financial Performance](./Report/ReportScopes#ifrs-17-financial-performance)** (profit & loss) require further calculations provided by ReportScopes.  An IFRS 17 financial performance report has at least four sections: Insurance Revenue, Insurance Service Expense, Insurance Finance Income/Expense and Other Comprehensive Income.  Formulas are provided [here](./Report/ReportScopes#ifrs-17-financial-performance).